<a href="https://colab.research.google.com/github/BrandonJChan/301FinalProject/blob/master/Data_Collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import requests
import json
from pandas.io.json import json_normalize
import time
import numpy as np
import re
from google.colab import files

In [2]:
!pip install spotipy

In [0]:
import spotipy
from spotipy import util
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.oauth2 as oauth2

#My 2019 Summer Streaming History

I requested my data from Spotify this past summer and this is my entire streaming history for the summer. I received a JSON data file that includes every song I listened to from June 6, 2019 to September 4, 2019, when I listened to every song, how long I listened to every song.

In [0]:
streaming_history = pd.read_json('https://github.com/BrandonJChan/301FinalProject/blob/master/Data/StreamingHistory.json?raw=true')

Getting rid of podcast listens from streaming history and songs I listened to for less than 1 second.

In [5]:
streaming_history = streaming_history[(streaming_history['msPlayed'] < 1000000)&
                                      (streaming_history['msPlayed'] > 1000)]
streaming_history

,endTime,artistName,trackName,msPlayed
0,2019-05-18 04:18,Seven Lions,A Way To Say Goodbye,131564
1,2019-06-06 00:03,Travis Scott,Pornography,231826
2,2019-06-06 00:11,Travis Scott,Oh My Dis Side (feat. Quavo),351253
3,2019-06-06 00:19,Travis Scott,3500 (feat. Future & 2 Chainz),349092
4,2019-06-06 00:23,Travis Scott,Wasted (feat. Juicy J),235680
...,...,...,...,...
16496,2019-09-04 23:11,Mac Ayres,Walking Home,61131
16497,2019-09-04 23:13,Bas,Jollof Rice (with EARTHGANG),128139
16498,2019-09-04 23:16,Bas,Fried Rice (with JID),175693
16499,2019-09-04 23:19,Bas,Nirvana (with Falcons & B. Lewis),165953


Getting the total amount of time spent listening to each song throughout the entire streaming history.

In [0]:
by_song = streaming_history.groupby(['trackName', 'artistName'])['msPlayed'].sum()
by_song_df = pd.DataFrame(by_song.sort_values(ascending=False)).reset_index()

Dropping songs that are no longer on spotify and the remaining podcasts.

In [0]:
by_song_df.drop([520, 756, 1174, 1201, 1334], inplace = True)

Fixing changed song titles so they are searchable

In [0]:
by_song_df['trackName'].replace(
    {'Awake (feat. JPEGMAFIA)': 'Awake',
     'Fried Rice (feat. JID)': 'Fried Rice (with JID)',
     'Swivel (with EARTHGANG) - From The Upcoming Album "Mirrorland"':'Swivel (with EARTHGANG)',
     '5% TINT': '5 TINT',
     'Trndsttr (feat. M. Maggie) - Lucian Remix': 'Trndsttr - Lucian Remix',
     'Kolors ft. Smino': 'Kolors',
     'Slow Down Love (feat. Chelsea Cutler)': "Slow Down Love",
     "Win You Over (feat. Soak)": "Win You Over",
     'Turn off the Lights (feat. Alexis Roberts)': 'Turn off the Lights',
     'On It! (feat. AZChike & 1TakeJay)': 'On It!',
     'LMK_WHAT’S REALLY GOOD REMIX_ FEAT_PRINCESS NOKIA_JUNGLEPUSSY_CUPCAKKE_MS. BOOGIE_100 BPM': 'LMK WHATS',
     'The West (feat. Kai Ca$h & 03 Greedo)': 'The West',
     'Surf (feat. City Girls, AZChike & Tee Grizzley)': 'Surf',
     'Alley Oop (feat. Lil Baby)': 'Alley Oop',
     'Monica Lewinsky (feat. A Boogie wit da Hoodie)': 'Monica Lewinsky',
     'How Do You Sleep? - Featuring Ludacris': 'How Do You Sleep? - Radio Edit Remix',
     'Good To Ya (feat. Ty Dolla $ign)': 'Good To Ya',
     'Naughty (feat. Jeremih)': 'Naughty'
     },
     inplace = True
)

In [9]:
by_song_df

,trackName,artistName,msPlayed
0,Say It (feat. Tove Lo),Flume,15502360
1,Freudian,Daniel Caesar,11709180
2,Feel Good (feat. Daya),Gryffin,10724803
3,Never Be Like You (feat. Kai),Flume,8174487
4,Get You (feat. Kali Uchis),Daniel Caesar,7469173
...,...,...,...
5041,Dancing With Our Hands Tied,Taylor Swift,1031
5042,Voices,Saosin,1021
5043,Come Here,Sabrina Claudio,1020
5044,Basement,DJ Mamba,1011


Getting access token so I can use the Spotify Web API

In [0]:
redirect_url = 'https://developer.spotify.com/documentation/web-api/'
access_token = util.prompt_for_user_token('chansthecrapper',
                           client_id='b933e8011ebe4d68a6dcb80fa40faa36',
                           client_secret='0f3629522e8249f5bca055bd2ab7c7a5',
                           redirect_uri=redirect_url)

In [0]:
search_endpoint = 'https://api.spotify.com/v1/search'
header = {'Authorization':'Bearer {}'.format(access_token)}

##Getting Spotify IDs for every song

I need the Spotify IDs to get the audio features (loudness, danceability, etc.) of every song. I am getting Spotify IDs by using the Spotify Web API to search for songs based on the song title and artist name, then I am taking the Spotify ID of the top result from the search. Also, extracting whether or not the song is explicit and the popularity of the song because they could potentially be helpful in the model.

In [0]:
def get_track(df):
  try:
    trackNameformatted = df['trackName'].replace(' ', '%20')
    trackNameformatted = re.sub("[#']", '', trackNameformatted)

    artistNameformatted = df['artistName'].replace(' ', '%20')

    search_response = requests.get(search_endpoint+
                                  "?q=track:{}%20artist:{}".format(trackNameformatted, artistNameformatted) + 
                                  "&limit=1&type=track",
                                  headers = header)
    time.sleep(0.2)
  
    track_row = json_normalize(json_normalize(search_response.json())['tracks.items'].iloc[0])

    artistName = json_normalize(track_row['artists'].iloc[0])['name'].iloc[0]

    final_row = pd.DataFrame({
        'trackName': [track_row['name'].iloc[0]],
        'artistName': [artistName],
        'explicit': [track_row['explicit'].iloc[0]*1],
        'popularity':[track_row['popularity'].iloc[0]],
        'id': [track_row['id'].iloc[0]]
    })
  except:
    final_row = pd.DataFrame({
        'trackName': [df['trackName']],
        'artistName': [df['artistName']],
        'explicit': ['unknown'],
        'popularity':[-1],
        'id': ['unknown']
    })
  return final_row

In [0]:
ids_series = by_song_df.apply(get_track, axis = 1)
ids_df = pd.concat(ids_series.values.tolist())

In [14]:
ids_df[ids_df['id'] == 'unknown'].head()

,trackName,artistName,explicit,popularity,id
0,Core - Original Mix,RL Grime,unknown,-1,unknown
0,Find,Kasbo,unknown,-1,unknown
0,Butterfly,LOOΠΔ,unknown,-1,unknown
0,Simple Solutions,TED Radio Hour,unknown,-1,unknown
0,Lie,Riah,unknown,-1,unknown


Getting rid of the 30-ish songs that I could not search for and remaining podcasts.

In [0]:
ids_df = ids_df[ids_df['id'] != 'unknown']

In [71]:
ids_df

,trackName,artistName,explicit,popularity,id
0,Say It (feat. Tove Lo),Flume,1,62,5aOpzm8W8zysk4asB9hxJw
0,Freudian,Daniel Caesar,1,54,4nESnAOALHmxg8UFCCtXhO
0,Feel Good (feat. Daya),Gryffin,0,72,0Ev562zA4pmUsBqjxsxxdx
0,Never Be Like You (feat. Kai),Flume,1,68,5WI2ltQIdwgzf1SNE76JyR
0,Get You (feat. Kali Uchis),Daniel Caesar,0,76,7zFXmv6vqI4qOt4yGf3jYZ
...,...,...,...,...,...
0,Prescription/Oxymoron,ScHoolboy Q,1,44,3ljBzccXOpkHcGejRzHVjK
0,Dancing With Our Hands Tied,Taylor Swift,0,62,7I7JbDv63ZJJsSi24DyJrz
0,Voices,Saosin,0,53,71U9X93wy4BQGyvmjFqApv
0,Come Here,Sabrina Claudio,0,49,4BTMGOsnbzI76XYTxAn5K8


##Getting Audio Features

I am only allowed to get audio features for, at most, 100 songs at a time. I am iterating through 100 IDs at a time and extracting the audio features using the Web API.

In [0]:
indices = list(range(0, len(ids_df), 100))
indices.append(len(ids_df))

In [0]:
features_endpoint = 'https://api.spotify.com/v1/audio-features/?ids='
feat_cols = ['id',
             'danceability',
             'energy',
             'key',
             'loudness',
             'mode',
             'speechiness',
             'acousticness',
             'instrumentalness',
             'liveness',
             'valence',
             'tempo',
             'duration_ms',
             'time_signature']
             
features_df = pd.DataFrame(columns = feat_cols)

Correcting one Spotify ID (It's Shake It Off by Taylor Swift)

In [0]:
ids_df['id'].iloc[870] = '5xTtaWoae3wi06K5WfVUUH'

In [0]:
for i in range(len(indices)-1):
  ids_subset = ids_df['id'].iloc[indices[i]:indices[i+1]].tolist()
  ids_string = ','.join(ids_subset)
  feat_response = requests.get(features_endpoint+
                              ids_string,
                              headers = header)

  feat_df = json_normalize(feat_response.json(), 'audio_features')[feat_cols]
  features_df = features_df.append(feat_df)
  time.sleep(0.2)

In [83]:
features_df

,id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,5aOpzm8W8zysk4asB9hxJw,0.598,0.532,3,-6.833,0,0.0295,0.066800,0.000003,0.0617,0.2610,74.963,262521,4
1,4nESnAOALHmxg8UFCCtXhO,0.355,0.298,8,-12.277,1,0.0581,0.310000,0.002660,0.1850,0.0714,115.823,602297,3
2,0Ev562zA4pmUsBqjxsxxdx,0.625,0.707,2,-4.761,1,0.0337,0.220000,0.000000,0.2130,0.4790,138.064,248156,4
3,5WI2ltQIdwgzf1SNE76JyR,0.444,0.557,0,-5.476,1,0.0602,0.453000,0.000000,0.1520,0.2960,115.058,234910,4
4,7zFXmv6vqI4qOt4yGf3jYZ,0.658,0.294,4,-8.533,0,0.0321,0.422000,0.000052,0.0749,0.3580,74.038,278180,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,3ljBzccXOpkHcGejRzHVjK,0.531,0.910,11,-5.397,0,0.4350,0.049500,0.008720,0.1050,0.3760,119.997,429667,4
81,7I7JbDv63ZJJsSi24DyJrz,0.624,0.691,0,-6.686,0,0.1960,0.060400,0.000011,0.1380,0.2840,160.024,211507,4
82,71U9X93wy4BQGyvmjFqApv,0.307,0.925,9,-3.918,0,0.0926,0.000094,0.009250,0.3770,0.2610,179.962,217525,4
83,4BTMGOsnbzI76XYTxAn5K8,0.551,0.517,11,-8.605,0,0.0499,0.826000,0.006590,0.3910,0.1990,140.020,242011,4


Joining the DataFrame with the IDs with the DataFrame with the audio features.

In [0]:
ids_merge = ids_df.merge(features_df,
                         left_on='id',
                         right_on='id')

Joining the DataFrame with the IDs and audio features with my streaming history DataFrame.

In [0]:
final_df = by_song_df.merge(ids_merge,
                            left_on = ['trackName', 'artistName'],
                            right_on = ['trackName', 'artistName'])

In [0]:
final_df = final_df.drop_duplicates()

Adjust for the length of each song by apporximating the total number of plays.

In [0]:
final_df['plays'] = final_df['msPlayed'] / final_df['duration_ms']

Ordering and filtering the columns.

In [0]:
final_df = final_df[['trackName',
                     'artistName',
                     'msPlayed',
                     'duration_ms',
                     'plays',
                     'explicit',
                     'popularity',
                     'danceability',
                     'energy',
                     'key',
                     'loudness',
                     'mode',
                     'speechiness',
                     'acousticness',
                     'instrumentalness',
                     'liveness',
                     'valence',
                     'tempo',
                     'time_signature']]

#The Final DataFrame (Training Set)

In [89]:
final_df.head()

,trackName,artistName,msPlayed,duration_ms,plays,explicit,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,Say It (feat. Tove Lo),Flume,15502360,262521,59.0519,1,62,0.598,0.532,3,-6.833,0,0.0295,0.0668,0.000003,0.0617,0.2610,74.963,4
1,Freudian,Daniel Caesar,11709180,602297,19.4409,1,54,0.355,0.298,8,-12.277,1,0.0581,0.3100,0.002660,0.1850,0.0714,115.823,3
2,Feel Good (feat. Daya),Gryffin,10724803,248156,43.218,0,72,0.625,0.707,2,-4.761,1,0.0337,0.2200,0.000000,0.2130,0.4790,138.064,4
3,Never Be Like You (feat. Kai),Flume,8174487,234910,34.7984,1,68,0.444,0.557,0,-5.476,1,0.0602,0.4530,0.000000,0.1520,0.2960,115.058,4
4,Get You (feat. Kali Uchis),Daniel Caesar,7469173,278180,26.8501,0,76,0.658,0.294,4,-8.533,0,0.0321,0.4220,0.000052,0.0749,0.3580,74.038,4


#Getting Tracks from my 'Release Radar' Playlist
I will be making predictions on songs that have recently been released and reccomended to me. I can use the predictions to see which ones I should listen to and which ones I shouldn't listen to.

In [0]:
release_endpoint = 'https://api.spotify.com/v1/playlists/'
params = '37i9dQZEVXbqSLjqzeB9BL/tracks?market=US&limit=15&offset=0'
response = requests.get(release_endpoint + params, headers = header)

In [0]:
def get_artist(json):
  return json_normalize(json)['name'].iloc[0]

In [0]:
tracks_df = json_normalize(json_normalize(response.json())['items'].iloc[0])
tracks_df['artistName'] = tracks_df['track.artists'].apply(get_artist)
tracks_df = tracks_df[['track.id', 'track.name', 'artistName', 'track.popularity', 'track.explicit']]

In [0]:
tracks_df.rename({
    'track.id': 'id',
    'track.name': 'trackName',
    'artistName': 'artistName',
    'track.popularity': 'popularity',
    'track.explicit': 'explicit'
},
axis = 1,
inplace = True)

tracks_df['explicit'] = tracks_df['explicit']*1

Getting the audio features for these songs.

In [0]:
feat_response = requests.get(features_endpoint+
                              ','.join(tracks_df['id'].tolist()),
                              headers = header)

feat_df = json_normalize(feat_response.json(), 'audio_features')[feat_cols]

In [0]:
final_test_df = tracks_df.merge(feat_df,
                                left_on = 'id',
                                right_on = 'id')[['trackName',
                                                  'artistName',
                                                  'explicit',
                                                  'popularity',
                                                  'danceability',
                                                  'energy',
                                                  'key',
                                                  'loudness',
                                                  'mode',
                                                  'speechiness',
                                                  'acousticness',
                                                  'instrumentalness',
                                                  'liveness',
                                                  'valence',
                                                  'tempo',
                                                  'time_signature'
                                                  ]]

In [98]:
final_test_df

,trackName,artistName,explicit,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,The Difference,Flume,0,72,0.560,0.865,1,-1.148,1,0.0909,0.25500,0.000153,0.0830,0.424,159.960,4
1,SUGAR (Remix) [feat. Dua Lipa],BROCKHAMPTON,0,75,0.661,0.523,8,-5.902,1,0.0381,0.48300,0.000000,0.4690,0.536,122.948,4
2,DIET_ - A COLORS SHOW,Denzel Curry,1,53,0.799,0.614,1,-10.165,1,0.5250,0.05210,0.000000,0.3670,0.844,90.880,4
3,Are You Feeling Sad?,Little Dragon,0,64,0.767,0.646,5,-8.612,0,0.0424,0.06160,0.000058,0.3330,0.462,113.006,4
4,FUN! - SILO x Martin Wave Remix,Vince Staples,0,51,0.766,0.898,10,-4.193,0,0.0561,0.02930,0.000336,0.0900,0.580,101.019,4
5,Don't Slack (from Trolls World Tour),Anderson .Paak,0,65,0.617,0.837,9,-5.574,1,0.1150,0.08420,0.000000,0.3440,0.800,168.120,4
6,Trapped In,REASON,1,49,0.739,0.626,1,-7.814,1,0.3410,0.02310,0.000000,0.1070,0.359,145.917,4
7,Something Comforting,Porter Robinson,0,65,0.470,0.560,8,-9.707,1,0.0298,0.04490,0.000004,0.0774,0.290,143.987,4
8,LEGACY,DPR LIVE,0,56,0.850,0.574,4,-6.253,0,0.2630,0.00735,0.000205,0.0908,0.243,149.980,4
9,tell me tell me,m-flo loves Sik-K & eill & 向井太一,0,54,0.753,0.730,3,-4.020,1,0.1670,0.26300,0.000000,0.0576,0.788,134.985,4
